In [1]:
"""
NEED TO BLOCK COMPONENTS AND PHANTOMS SINCE THE 1020 3/4 IS NOW THE
1020 5/6
NEED TO UPDATE GXS WITH NEW NUMBERS AND UPDATED PUMP DESCRIPTION

THERE ARE 4 DIFFERENCES WITH THE CASE CHANGE
1- NO PERFORMANCE CHANGE
2- APPEARNACE CHANGE BECAUSE OF THE ADDED RIBS WHICH CHANGED 2 OTHER
COMPONENTS
3- 4 LONGER CAPSCREWS
4- SLIGHT CHANGE TO THE GASKET WITH THE SHIFT IN THE BOLT HOLES
NO CHANGE TO EXISTING KITS- REVISIONS WERE MADE TO THESE BOMS

1020 CASING (BUY COMPONENT) CHANGE KP
125# 91840054 CHANGED TO 99717392       # Not Found
250# 91840056 CHANGED TO 99717485       # Not Found

1020 PHANTOM CHANGE
250# 98441969 CHANGED TO 92758523       # Not Found
125# 96698939 CHANGED TO 92758528       # in KPbom.xlsx 

HARDWARE PHANTOM CHANGED
KP
GRADE 5 96698968 CHANGE TO 92758121     # in KPbom.xlsx
GRADE 8 96768950 CHANGE TO 92758767     # in KPbom.xlsx
KPVS
GRADE 5 99413027 CHANGE TO 92758120     # Not Found
GRADE 8 NEW NUMBER         92758769     # ??

BOMS THAT HAVE BEEN CHANGED ARE IN THE FILE
"""

'\nNEED TO BLOCK COMPONENTS AND PHANTOMS SINCE THE 1020 3/4 IS NOW THE\n1020 5/6\nNEED TO UPDATE GXS WITH NEW NUMBERS AND UPDATED PUMP DESCRIPTION\n\nTHERE ARE 4 DIFFERENCES WITH THE CASE CHANGE\n1- NO PERFORMANCE CHANGE\n2- APPEARNACE CHANGE BECAUSE OF THE ADDED RIBS WHICH CHANGED 2 OTHER\nCOMPONENTS\n3- 4 LONGER CAPSCREWS\n4- SLIGHT CHANGE TO THE GASKET WITH THE SHIFT IN THE BOLT HOLES\nNO CHANGE TO EXISTING KITS- REVISIONS WERE MADE TO THESE BOMS\n\n1020 CASING (BUY COMPONENT) CHANGE KP\n125# 91840054 CHANGED TO 99717392       # Not Found\n250# 91840056 CHANGED TO 99717485       # Not Found\n\n1020 PHANTOM CHANGE\n250# 98441969 CHANGED TO 92758523       # Not Found\n125# 96698939 CHANGED TO 92758528       # in KPbom.xlsx \n\nHARDWARE PHANTOM CHANGED\nKP\nGRADE 5 96698968 CHANGE TO 92758121     # in KPbom.xlsx\nGRADE 8 96768950 CHANGE TO 92758767     # in KPbom.xlsx\nKPVS\nGRADE 5 99413027 CHANGE TO 92758120     # Not Found\nGRADE 8 NEW NUMBER         92758769     # ??\n\nBOMS THAT H

In [2]:
import os
import openpyxl
import pandas as pd

In [3]:
partnumbers_to_change = {
    '91840054':'99717392',
    '91840056':'99717485',
    '98441969':'92758523',
    '96698939':'92758528',     # in KPbom.xlsx 
    '96698968':'92758121',     # in KPbom.xlsx
    '96768950':'92758767',     # in KPbom.xlsx
    '99413027':'92758120',
    '1020-3/4':'1020-5/6',
    '\1020-3_4':'1020-5_6',
    'Desc-1020-3_4-KP_4':'Desc-1020-5_6-KP_6',
    'Desc-1020-3_4-KP_3':'Desc-1020-5_6-KP_5'
}

In [4]:
myDir = r'C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\15 KP-KPV-KPVS Updates\PSDs'

for root, dirs, files in os.walk(myDir):
    for filename in files:
        filePath = os.path.join(myDir, filename)
        # print(filePath)
        
        wb = openpyxl.load_workbook(filePath)

        for sheet in wb.worksheets:
            if sheet.title == 'Case':   # Iterate through PSDs andd other tabs
                    
                # Read in Excel sheet to a dataframe
                sheetname = sheet.title
                data = pd.read_excel(filePath, sheet_name=sheetname)
                data = data.astype(str)

                for old_pn, new_pn in partnumbers_to_change.items():
                    # Check if old partnumbers exist in DataFrame
                    if old_pn in data.values:
                        print(f'old_pn: {old_pn} exists in PSD: {filename}, sheet: {sheetname}')
                        
                        print(f'old_pn type: {old_pn.__class__}, match_value type: {type(data.iloc[177,0])}')
                        data.replace(to_replace=old_pn, value=new_pn, inplace=True, regex=True)

                        print(data)
            
                        # Fix formatting
                        from openpyxl.styles import PatternFill
                        for rows in sheet.iter_rows(min_row=1, max_row=6, min_col=1, max_col=40):
                            for cell in rows:
                                cell.fill = PatternFill(start_color="FFAE42", end_color="FFAE42", fill_type="solid")

                        wb.save(filePath)

                        ### Writing to excel file
                        # Write resulting dataframes to new sheet in PSD with changes.
                        psd_startrow = 1
                        psd_startcol = 1

                        with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:  
                            data.to_excel(writer, sheet_name=sheetname, index=False, startrow=psd_startrow, startcol=psd_startcol)
                            # removals.to_excel(writer, sheet_name=tabName, index=False, header=False, startrow=append_location+1, startcol=psd_startcol)

                        # Remove original tab, replace with new one?
